In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns
from tqdm import tqdm

import sys

sys.path.insert(0, r'c:\Users\joneh\master_thesis\src_HF')
from utils.main_utils import *

import warnings

warnings.filterwarnings("ignore")
plt.style.use('science')


In [ ]:
df_LCOc1 = pd.read_csv(r'C:\Users\joneh\master_thesis\data\time_series\LCOc1_High_Frequency.csv') # Brent Crud Oil
df_CLc1 = pd.read_csv(r'C:\Users\joneh\master_thesis\data\time_series\CLc1_High_Frequency.csv') # WTI Crud Oil

# create datetime index
df_LCOc1.index = pd.to_datetime(df_LCOc1['Date'])
df_CLc1.index = pd.to_datetime(df_CLc1['Date'])

config = load_json(r'C:\Users\joneh\master_thesis\src_HF\plot_config.json')

### Price plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
df_LCOc1['CLOSE'].plot(label='ICE Brent Crude', ax=ax, lw=0.8)
df_CLc1['CLOSE'].plot(label='NYMEX Light Sweet', ax=ax, lw=0.8)
ax.set_ylabel('Price (USD)', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.legend(frameon=False, loc='upper left', fontsize=13)
ax.tick_params(axis='x', labelsize=14)

fig.savefig('images/crude_oil_price.png', bbox_inches='tight', dpi=200)

### Volume plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# create a stacked bar plot resampled bby yearly frequency
volume_df = pd.concat([df_LCOc1['VOLUME'].resample('w').sum(), df_CLc1['VOLUME'].resample('w').sum()], axis=1)
volume_df.columns = ['ICE Brent Crude', 'NYMEX Light Sweet']

volume_df.plot(kind='bar', stacked=True, ax=ax)

ax.set_ylabel('Volume', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.legend(frameon=False, loc='upper left', fontsize=13)
ax.tick_params(axis='x', labelsize=14)

# fix x-tick labels to show month and year and only show every 4th label and .strftime('%Y-%m')
plt.xticks(
    np.arange(0, len(volume_df), 4), 
    volume_df.index[np.arange(0, len(volume_df), 4)].strftime('%Y-%m'), 
    rotation=45
)


fig.savefig('images/crude_oil_volume.png', bbox_inches='tight', dpi=200)

In [ ]:
df_CLc1['CLOSE'].iloc[-50000:].plot(figsize=(12,6))

In [ ]:
df_ = df_LCOc1.copy()

result_dict = {}

df_['LOGRET'] = np.log(df_['CLOSE']).diff()

days = df_.index.normalize().unique()

for i in days:

    date = i.date()

    df = df_[str(date)]['LOGRET']

    df.index = pd.Index([time.time() for time in df.index], name='Time')

    result_dict[str(date)] = df



In [ ]:
df = pd.DataFrame(result_dict)

mean = df.mean(axis=1)

fig, ax = plt.subplots(figsize=(12,6))
ax2 = ax.twinx()

mean.cumsum().plot(ax=ax2, lw=1.5, color='black')

for i in df.columns:
    df[i].cumsum().plot(ax=ax, lw=0.3)